- Buộc phải cài accelerate để không có lỗi : RuntimeError: Input tensors need to be on the same GPU, but found the following tensor and device combinations
- Buộc phải -U trước khi đi vào các step sau

In [1]:
# Install libs
! pip install -qq peft==0.15.2  trl==0.17.0 
# bitsandbytes==0.45.5
! pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires

#### Thêm hàng này để tránh chạy multi gpu

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print("CUDA devices =", torch.cuda.device_count())   # Phải in ra 1
print("Current device =", torch.cuda.current_device())  # Phải là 0

CUDA devices = 1
Current device = 0


In [3]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig, SFTConfig, SFTTrainer
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import wandb

from peft import LoraConfig

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Preference Tuning with LLMs")
secret_value_1 = user_secrets.get_secret("push_to_hub")

wandb.login(key=secret_value_0)

import warnings
warnings.filterwarnings("ignore")

2025-05-09 08:14:45.335779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746778485.759158      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746778485.879200      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moonlig73 (minhdeptrai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
CACHE_DIR = "/kaggle/working/"

dataset = load_dataset("thainq107/Vi-Alpaca-Preference", cache_dir=CACHE_DIR)
dataset_train = dataset["train"].select(range(50))
dataset_train[0]

README.md:   0%|          | 0.00/506 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/61.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65017 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'id': 'alpaca-7294',
 'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
 'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
 'rejected': 'Câu này không có lỗi ngữ pháp.'}

In [5]:
LORA_RANK        = 8
TARGET_MODULES   = [
    "q_proj","k_proj","v_proj","o_proj",
    "gate_proj","up_proj","down_proj",
]
MAX_SEQ          = 1024
peft_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_RANK,
    target_modules=TARGET_MODULES,
    task_type="CAUSAL_LM",
)

In [6]:
# Model to fine-tune
model_name = "TSunm/Llama-3.2-1B-Instruct-sft"
cache_dir = "/kaggle/working/"

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name,
#     max_seq_length=MAX_SEQ,
#     dtype=torch.float16,
#     load_in_4bit=False,
#     device_map={"": torch.cuda.current_device()},
# )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map={'':torch.cuda.current_device()},
    cache_dir=cache_dir,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)
base_model.config.use_cache = False
base_model

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=512, bias=False)
            (lora_dropout): ModuleDict(
         

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir=cache_dir, trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [8]:
def convert_to_conversational_preference_format(example):
    return {
        "id": example["id"],
        "prompt": [{"role": "system",
                    "content": "You are a helpful assistant."},
                   {"role": "user",
                    "content": example["question"]}],
        "chosen": [{"role": "assistant",
                    "content": example["chosen"]}],
        "rejected": [{"role": "assistant",
                      "content": example["rejected"]}],
    }

dpo_dataset = dataset_train.map(convert_to_conversational_preference_format)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [9]:
hyperparameters = {
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 1,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 1,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "fp16": True
}


In [10]:
DPO_OUTPUT_DIR = f"/kaggle/working/Llama-3.2-1B-Instruct-dpo"
dpo_args = DPOConfig(
    **{ **hyperparameters, "output_dir":
       DPO_OUTPUT_DIR, "max_length": MAX_SEQ }
)

dpo_trainer = DPOTrainer(
    base_model,
    args=dpo_args,
    train_dataset=dpo_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)
dpo_trainer.train()

Extracting prompt in train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250509_081539-splxql4v
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run /kaggle/working/Llama-3.2-1B-Instruct-dpo
wandb: ⭐️ View project at https://wandb.ai/minhdeptrai/huggingface
wandb: 🚀 View run at https://wandb.ai/minhdeptrai/huggingface/runs/splxql4v


Step,Training Loss


TrainOutput(global_step=50, training_loss=0.6901610565185546, metrics={'train_runtime': 44.2331, 'train_samples_per_second': 1.13, 'train_steps_per_second': 1.13, 'total_flos': 0.0, 'train_loss': 0.6901610565185546, 'epoch': 1.0})

In [11]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(
    repo_id="TSunm/Llama-3.2-1B-Instruct-dpo-vi-alpaca",
    token=secret_value_1,
    repo_type="model",
    exist_ok=True 
)

RepoUrl('https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-dpo-vi-alpaca', endpoint='https://huggingface.co', repo_type='model', repo_id='TSunm/Llama-3.2-1B-Instruct-dpo-vi-alpaca')

In [12]:
dpo_trainer.model.push_to_hub(
    repo_id=model_name,
    token=secret_value_1
)
# push tokenizer lên cùng
tokenizer.push_to_hub(
    repo_id=model_name,
    token=secret_value_1
)

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-sft/commit/b5e0578c476986b780c909b6edf4ac0a483bdc6f', commit_message='Upload tokenizer', commit_description='', oid='b5e0578c476986b780c909b6edf4ac0a483bdc6f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='TSunm/Llama-3.2-1B-Instruct-sft'), pr_revision=None, pr_num=None)